In [1]:
import ast
import numpy as np
import os
import pandas as pd
import pickle
import warnings

import matplotlib
matplotlib.use('Agg')  # Use the Agg backend which does not require a windowing system
import matplotlib.pyplot as plt

import wandb

from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE, RandomOverSampler
from lightgbm import LGBMClassifier
from pyHSICLasso import HSICLasso
from scipy.signal import find_peaks, savgol_filter
from scipy.sparse import diags, eye
from scipy.sparse.linalg import spsolve
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, ElasticNet
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay, make_scorer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# Ignore all warnings
warnings.filterwarnings('ignore')

In [2]:
class HSIC_Lasso_custom():

    def __init__(self, num_features=200):
        self.num_features = num_features
        self.hsic_lasso = None
        self.selected_features = None
    
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        featname = list(np.arange(X.shape[1]))
        self.hsic_lasso = HSICLasso()
        self.hsic_lasso.input(X, y, featname=featname)
        self.hsic_lasso.regression(num_feat=self.num_features, M=5, n_jobs=-1)
        self.selected_features = self.hsic_lasso.get_index()
        self.selected_features = np.array(self.selected_features, dtype=int)

    def transform(self, X):
        return X.iloc[:, self.selected_features]
    
class Lasso_custom():
        def __init__(self, alpha=0.1):
            self.alpha = alpha
            self.lasso = None
            self.selected_features = None
        
        def fit(self, X, y):
            X = np.array(X)
            y = np.array(y)
            self.lasso = Lasso(alpha=self.alpha)
            self.lasso.fit(X, y)
            self.selected_features = np.where(self.lasso.coef_ != 0)[0]
        
        def transform(self, X):
            return X.iloc[:, self.selected_features]

In [3]:
def plot_confusion_matrix(y_true, y_pred, labels, title):
    """
    Plots a confusion matrix as a heatmap.

    Parameters:
    y_true (numpy.ndarray): True labels.
    y_pred (numpy.ndarray): Predicted labels.
    labels (list): List of class labels.
    title (str): Title of the plot.
    """
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels).plot(cmap='Blues', ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')
    
    # Log the plot directly to wandb
    wandb.log({"Confusion Matrix": wandb.Image(fig)})
    
    # Close the plot to free up memory
    plt.close(fig)

In [4]:
def model_train_evaluate(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        # Load the dataset
        with open('datasets.pkl', 'rb') as f:
            datasets = pickle.load(f)
        
        # Access the dataset for the specified combination
        key = (config.oversampling_method, config.feature_selection_method)
        dataset = datasets.get(key)

        if not dataset:
            print(f"No dataset found for combination: {key}")
            return

        X_train = dataset['X_train']
        y_train = dataset['y_train']
        X_test = dataset['X_test']
        y_test = dataset['y_test']
        
        # Model initialization
        models = {
            'LogisticRegression': LogisticRegression(max_iter=1000),
            'DecisionTree': DecisionTreeClassifier(),
            'RandomForest': RandomForestClassifier(),
            'GradientBoosting': GradientBoostingClassifier(),
            'LightGBM': LGBMClassifier(),
            'XGBoost': XGBClassifier(),
            'CatBoost': CatBoostClassifier(silent=True),
            'SVC': SVC()
        }
        
        # Cross-validation setup
        model = models[config.model]
        cv = StratifiedKFold(n_splits=5)
        scoring = {
            'accuracy': make_scorer(accuracy_score),
            'precision': make_scorer(precision_score, average='weighted'),
            'recall': make_scorer(recall_score, average='weighted'),
            'f1_weighted': make_scorer(f1_score, average='weighted'),
            'balanced_accuracy': make_scorer(balanced_accuracy_score)
        }
        results = cross_validate(model, X_train, y_train, cv=cv, scoring=scoring)
        
        # Model training
        model.fit(X_train, y_train)
        
        # Model evaluation
        y_pred = model.predict(X_test)
        test_accuracy = accuracy_score(y_test, y_pred)
        test_precision = precision_score(y_test, y_pred, average='weighted')
        test_recall = recall_score(y_test, y_pred, average='weighted')
        test_f1_score = f1_score(y_test, y_pred, average='weighted')
        test_balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        
        # Logging results with wandb
        wandb.log({
            'cv_f1-score': results['test_f1_weighted'].mean(),
            'cv_accuracy': results['test_accuracy'].mean(),
            'cv_precision': results['test_precision'].mean(),
            'cv_recall': results['test_recall'].mean(),
            'cv_balanced_accuracy': results['test_balanced_accuracy'].mean(),
            'test_accuracy': test_accuracy,
            'test_precision': test_precision,
            'test_recall': test_recall,
            'test_f1_score': test_f1_score,
            'test_balanced_accuracy': test_balanced_accuracy
        })
        
        # Plotting confusion matrix
        plot_confusion_matrix(y_test, y_pred, labels=['Not Surviving', 'Surviving'], title=f'{config.model}')


In [5]:
sweep_config = {
    'method': 'random',
    'name' : 'todos_random',
    'metric': {
        'name': 'test_balanced_accuracy',
        'goal': 'maximize'   
    },
    'parameters': {
        'oversampling_method': {
            'values': ['SMOTE', 'RandomOverSampler']
        },
        'feature_selection_method': {
            'values': ['PCA', 'Lasso', 'ElasticNet', 'HSICLasso', 'SelectKBest', 'SelectFromLinearRegression', 'SelectFromRandomForest']
        },
        'model': {
            'values': ['RandomForest', 'SVC', 'LogisticRegression', 'DecisionTree', 'GradientBoosting', 'XGBoost', 'CatBoost', 'LightGBM']
        }
    }
}

In [6]:
wandb.login()
sweep_id = wandb.sweep(sweep_config, project='practice2_salud')
wandb.agent(sweep_id, model_train_evaluate, count=400) 

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: diegocollado (practica_maldi). Use `wandb login --relogin` to force relogin


Create sweep with ID: oldshqv1
Sweep URL: https://wandb.ai/practica_maldi/practice2_salud/sweeps/oldshqv1


wandb: Agent Starting Run: sjnm5kth with config:
wandb: 	feature_selection_method: SelectFromLinearRegression
wandb: 	model: DecisionTree
wandb: 	oversampling_method: RandomOverSampler
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


cv_accuracy,▁
cv_balanced_accuracy,▁
cv_f1-score,▁
cv_precision,▁
cv_recall,▁
test_accuracy,▁
test_balanced_accuracy,▁
test_f1_score,▁
test_precision,▁
test_recall,▁
cv_accuracy,0.93003


...

cv_accuracy,▁
cv_balanced_accuracy,▁
cv_f1-score,▁
cv_precision,▁
cv_recall,▁
test_accuracy,▁
test_balanced_accuracy,▁
test_f1_score,▁
test_precision,▁
test_recall,▁
cv_accuracy,0.93317


wandb: Agent Starting Run: 2vforv2p with config:
wandb: 	feature_selection_method: Lasso
wandb: 	model: GradientBoosting
wandb: 	oversampling_method: SMOTE
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


cv_accuracy,▁
cv_balanced_accuracy,▁
cv_f1-score,▁
cv_precision,▁
cv_recall,▁
test_accuracy,▁
test_balanced_accuracy,▁
test_f1_score,▁
test_precision,▁
test_recall,▁
cv_accuracy,0.92495


wandb: Agent Starting Run: lfl3kxrx with config:
wandb: 	feature_selection_method: ElasticNet
wandb: 	model: CatBoost
wandb: 	oversampling_method: SMOTE
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


cv_accuracy,▁
cv_balanced_accuracy,▁
cv_f1-score,▁
cv_precision,▁
cv_recall,▁
test_accuracy,▁
test_balanced_accuracy,▁
test_f1_score,▁
test_precision,▁
test_recall,▁
cv_accuracy,0.66771
